# Homework2: Convolution Neural Network for classification

In this homework, we are going to learn:
1. How to preprocess and load data in pytorch
2. How to build a CNN model for training classification
3. Training/Validation Process and plot the result.
4. Testing Inference.

<p align="center">
<img src="https://miro.medium.com/max/895/1*RjZe7cfnhdRhhLimLvapow.png" width="800">
</p>

## 0.1 Preparation

In [ ]:
import os
import csv
import torch
import numpy as np
import pandas as pd

from PIL import Image
from google.colab import drive

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Use device:",device)

Use device: cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##############################################
# Decide your work and save path
##############################################
DataPath = '/content/hw2_ex/'
SavePath = '/content/drive/MyDrive/Colab Notebooks/homework2/'

os.makedirs(DataPath, exist_ok = True)
os.makedirs(SavePath, exist_ok = True)

## 0.2 Download DataSet: Cat and Dog

<p align="center">
<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3362/media/woof_meow.jpg" width="400">
</p>

In [ ]:
if not os.path.isdir(os.path.join(DataPath, 'train_dataset/')):
    !pip install --upgrade --no-cache-dir gdown
    !gdown --id 1hj2zrZI3Nd-C6nlGOE1crgR_gnpoKHQh --output 'dataset.zip'
    !unzip -q dataset.zip -d '/content/hw2_ex' # the -d should be the same as DataPath

else:
    print("File already exists.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=c5605e12a17f816b2d35b0663667ba4d0095c3b0dcfef8f4d151bf30010abe09
  Stored in directory: /tmp/pip-ephem-wheel-cache-n0ja5j93/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1HguaSxM4tMCGU0VjmWW0cCzf3EhoaOWr
To: /content/dataset.zip
100% 612M/612M [00:09<00:00, 66.1MB/s]


# 1 Data Preprocess
In this chapter, we will learn how to preprocess image data.

Using pandas to preprocess, it is a good module for data analytic.

Using dataset and dataloader from pytorch to setup training dataflow.

## 1.1 Pandas DataFrame
>[documentation](https://pandas.pydata.org/docs/index.html)
>
>Pandas is a Python library used for working with data sets.
It has functions for analyzing, cleaning, exploring, and manipulating data.
>
>The name "Pandas" has a reference to both "Panel Data", and "Python Data Analysis" and was created by Wes McKinney in 2008.


<p align="center">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/1200px-Pandas_logo.svg.png" width="600">
</p>

In [ ]:
import pandas as pd

df = pd.read_csv(os.path.join(DataPath, 'train.csv'))
df.describe()

,img,label
count,20000,20000
unique,20000,2
top,0.jpg,cat
freq,1,10000


In [ ]:
df.head()

,img,label
0,0.jpg,cat
1,1.jpg,cat
2,2.jpg,cat
3,3.jpg,cat
4,4.jpg,cat


In [ ]:
df['label']=='cat'

0         True
1         True
2         True
3         True
4         True
         ...  
19995    False
19996    False
19997    False
19998    False
19999    False
Name: label, Length: 20000, dtype: bool

In [ ]:
df['img'][df['label']=='dog']

10000    10000.jpg
10001    10001.jpg
10002    10002.jpg
10003    10003.jpg
10004    10004.jpg
           ...    
19995    19995.jpg
19996    19996.jpg
19997    19997.jpg
19998    19998.jpg
19999    19999.jpg
Name: img, Length: 10000, dtype: object

In [ ]:
df.at[10,'img']

'10.jpg'

In [ ]:
df['imgpath'] = DataPath + df['img'].astype(str)
df.head()

,img,label,imgpath
0,0.jpg,cat,/content/hw2_ex/0.jpg
1,1.jpg,cat,/content/hw2_ex/1.jpg
2,2.jpg,cat,/content/hw2_ex/2.jpg
3,3.jpg,cat,/content/hw2_ex/3.jpg
4,4.jpg,cat,/content/hw2_ex/4.jpg


In [ ]:
Class = {'cat':0, 'dog':1}
df['category'] = df['label'].astype('category').cat.codes
df

,img,label,imgpath,category
0,0.jpg,cat,/content/hw2_ex/0.jpg,0
1,1.jpg,cat,/content/hw2_ex/1.jpg,0
2,2.jpg,cat,/content/hw2_ex/2.jpg,0
3,3.jpg,cat,/content/hw2_ex/3.jpg,0
4,4.jpg,cat,/content/hw2_ex/4.jpg,0
...,...,...,...,...
19995,19995.jpg,dog,/content/hw2_ex/19995.jpg,1
19996,19996.jpg,dog,/content/hw2_ex/19996.jpg,1
19997,19997.jpg,dog,/content/hw2_ex/19997.jpg,1
19998,19998.jpg,dog,/content/hw2_ex/19998.jpg,1


In [ ]:
df.iloc[0]

img                         0.jpg
label                         cat
imgpath     /content/hw2_ex/0.jpg
category                        0
Name: 0, dtype: object

## 1.2 Preprocess data to list

We compare two kinds of way for preprocessing.
We can find that using the same framework for preprocessing would be efficient.

In [ ]:
##############################################
# Create a list for downloaded files,
#  including the path of the image 
#  and the corresponding label.
##############################################
import time


train_img_path = os.path.join(DataPath, 'train_dataset/')
df = pd.read_csv(os.path.join(DataPath, 'train.csv'))

# small experiment
start_time = time.time()

Class = {'cat':0, 'dog':1}
imgpath = []
imglabel = []

# Load csv and add path
for idx, row in df.iterrows():
    imgpath.append(os.path.join(train_img_path, str(row["img"])))
    imglabel.append(Class[row["label"]])
print(f"Process with for loop in {(time.time()-start_time):.3f} s\n\n")

idx = np.random.randint(len(imgpath), size=10)
print(np.array(imgpath)[idx])
print(np.array(imglabel)[idx])

Process with for loop in 1.695 s


['/content/hw2_ex/train_dataset/4846.jpg'
 '/content/hw2_ex/train_dataset/17846.jpg'
 '/content/hw2_ex/train_dataset/19308.jpg'
 '/content/hw2_ex/train_dataset/3684.jpg'
 '/content/hw2_ex/train_dataset/18694.jpg'
 '/content/hw2_ex/train_dataset/8175.jpg'
 '/content/hw2_ex/train_dataset/10817.jpg'
 '/content/hw2_ex/train_dataset/7344.jpg'
 '/content/hw2_ex/train_dataset/19342.jpg'
 '/content/hw2_ex/train_dataset/17161.jpg']
[0 1 1 0 1 0 1 0 1 1]


In [ ]:
# small experiment
start_time = time.time()
imgpath = (train_img_path + df['img'].astype(str)).tolist()
imglabel = df['label'].astype('category').cat.codes.tolist()
print(f"Process with pandas framework in {(time.time()-start_time):.3f} s\n\n")

print(np.array(imgpath)[idx])
print(np.array(imglabel)[idx])

Process with pandas framework in 0.013 s


['/content/hw2_ex/train_dataset/4846.jpg'
 '/content/hw2_ex/train_dataset/17846.jpg'
 '/content/hw2_ex/train_dataset/19308.jpg'
 '/content/hw2_ex/train_dataset/3684.jpg'
 '/content/hw2_ex/train_dataset/18694.jpg'
 '/content/hw2_ex/train_dataset/8175.jpg'
 '/content/hw2_ex/train_dataset/10817.jpg'
 '/content/hw2_ex/train_dataset/7344.jpg'
 '/content/hw2_ex/train_dataset/19342.jpg'
 '/content/hw2_ex/train_dataset/17161.jpg']
[0 1 1 0 1 0 1 0 1 1]


## 1.3 Pytorch Dataset & Dataloaders
>[documentation](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
>
>Code for processing data samples can get messy and hard to maintain; we ideally want our dataset code to be decoupled from our model training code for better readability and modularity. PyTorch provides two data primitives:  `torch.utils.data.DataLoader`  and  `torch.utils.data.Dataset`  that allow you to use pre-loaded datasets as well as your own data. 
>
>Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.


<p align="center">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/PyTorch_logo_black.svg/2560px-PyTorch_logo_black.svg.png" width="600">
</p>

### 1.3.1 torchvision.transforms

>[documentation](https://pytorch.org/vision/stable/transforms.html)
>
>`torchvision.transforms` help us to transform image to tensor and can also help us apply augmentation, for more robust training.

<p align="center">
<img src="https://pytorch.org/vision/stable/_images/sphx_glr_plot_transforms_024.png" width="600">
</p>


In [ ]:
# you need change image to tensor
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    ])
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    ])

### 1.3.2 torch.utils.data.Dataset

In [ ]:
##############################################
# Use the list you created above
#  to create a class for DataLoader.
##############################################
from torch.utils.data import Dataset, DataLoader

class dataset(Dataset):
    def __init__(self, imgpath, csvpath, transform = valid_transform):
        # --------------------------------------------
        # Initialize paths, transforms, and so on
        # --------------------------------------------
        df = pd.read_csv(csvpath)
        self.images = (imgpath + df['img'].astype(str)).tolist()
        self.label = df['label'].astype('category').cat.codes.tolist()
        self.transform = transform
        
    def __getitem__(self, index):
        # --------------------------------------------
        # 1. Read from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data (e.g. image and label)
        # --------------------------------------------
        imgpath = self.images[index]
        img = Image.open(imgpath).convert('RGB')
        label = self.label[index]
        img = self.transform(img)

        return img, label
        
    def __len__(self):
        # --------------------------------------------
        # Indicate the total size of the dataset
        # --------------------------------------------
        return len(self.images)


In [ ]:
from torch.utils.data import random_split

imgpath = os.path.join(DataPath, 'train_dataset/')
csvpath = os.path.join(DataPath, 'train.csv')

trainset = dataset(imgpath, csvpath)
train_set_size = int(len(trainset) * 0.7)
valid_set_size = int(len(trainset) * 0.15)
test_set_size = len(trainset) - train_set_size - valid_set_size

trainset, validset, testset = random_split(trainset, [train_set_size, valid_set_size, test_set_size])

trainset.transform = train_transform
validset.transform = valid_transform
testset.transform = valid_transform

print(f'trainset: {len(trainset)}\nvalidset: {len(validset)}\ntestset: {len(testset)}')

idx = np.random.randint(len(trainset))
print(f'{idx:5d}/{len(trainset)} : {trainset[idx]}')

trainset: 14000
validset: 3000
testset: 3000
 9108/14000 : (tensor([[[0.3451, 0.3804, 0.4863,  ..., 0.5608, 0.7176, 0.6784],
         [0.3294, 0.3882, 0.5647,  ..., 0.5137, 0.6706, 0.5529],
         [0.3451, 0.3882, 0.5725,  ..., 0.4980, 0.5216, 0.4314],
         ...,
         [0.3333, 0.3725, 0.5294,  ..., 0.5020, 0.7020, 0.6824],
         [0.3020, 0.3804, 0.5608,  ..., 0.6157, 0.7333, 0.8118],
         [0.3216, 0.3373, 0.5647,  ..., 0.4667, 0.7137, 0.7569]],

        [[0.3059, 0.3490, 0.4745,  ..., 0.5882, 0.7412, 0.7020],
         [0.2902, 0.3647, 0.5608,  ..., 0.5529, 0.7098, 0.5922],
         [0.3176, 0.3686, 0.5725,  ..., 0.5608, 0.5843, 0.4902],
         ...,
         [0.3765, 0.4118, 0.5725,  ..., 0.4353, 0.6510, 0.6510],
         [0.3373, 0.4157, 0.5961,  ..., 0.5608, 0.6824, 0.7647],
         [0.3529, 0.3647, 0.5882,  ..., 0.4392, 0.6667, 0.6980]],

        [[0.3294, 0.4078, 0.5765,  ..., 0.3255, 0.4902, 0.4667],
         [0.3137, 0.4235, 0.6667,  ..., 0.3020, 0.4706, 0.3725]

### 1.3.3 torch.utils.data.DataLoader

In [ ]:
# Loaded Datasets to DataLoaders

# batch_size also affect training step.
# higher: faster, stable
#   but inprecise on optimize, may use large memory

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers = 2)
validloader = DataLoader(validset, batch_size=64, shuffle=False, num_workers = 2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers = 2)

# 2 CNN Model

### 2.1 Convolutional layers
>[document](https://pytorch.org/docs/stable/nn.html#convolution-layers)
>
>Start to build our first CNN Model!

In [ ]:
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=3),
    )

result = model(torch.rand((64 ,3, 28, 28)))
print(result.shape)

torch.Size([64, 32, 8, 8])


In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(in_channels=16, out_channels=64, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    )
result = model(torch.rand((3, 3, 512, 1024)))
print(result.shape)

torch.Size([3, 128, 60, 124])


<p align="center">
<img src="https://1.bp.blogspot.com/-db1Bv-YnKTo/XkrIvCYFtEI/AAAAAAAAArA/2b0zbD29CQML4IKxp_1Zng_r0ioaOCZkgCEwYBhgL/s1600/appendix_C_conv_formula.png" width="600">
</p>


In [ ]:
data_flow = torch.rand((3, 3, 512, 1024))
print(data_flow.shape)

data_flow = nn.Conv2d(3,16,5)(data_flow)
print(data_flow.shape)

data_flow = nn.MaxPool2d(2,2)(data_flow)
print(data_flow.shape)

data_flow = nn.Conv2d(16,64,5)(data_flow)
print(data_flow.shape)

data_flow = nn.MaxPool2d(2,2)(data_flow)
print(data_flow.shape)

data_flow = nn.Conv2d(64,128,5)(data_flow)
print(data_flow.shape)

print((data_flow.shape))

torch.Size([3, 3, 512, 1024])
torch.Size([3, 16, 508, 1020])
torch.Size([3, 16, 254, 510])
torch.Size([3, 64, 250, 506])
torch.Size([3, 64, 125, 253])
torch.Size([3, 128, 121, 249])
torch.Size([3, 128, 121, 249])


### 2.2 Convolutional model

Try to implement LeNet-5 with pytorch !

(but using `nn.ReLU()` as activation function)

<p align="center">
<img src="https://cdn-images-1.medium.com/max/1024/1*DMcPgeekUftwk0GTMcNawg.png" width="900">
</p>

In [ ]:
##############################################
# Build your model here!
# 
# Practice:
#   Try to implement LeNet-5 with pytorch !
##############################################
class trainmodel(nn.Module):
    def __init__(self):
        super(trainmodel, self).__init__()
        self. conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            )
        self.cls = nn.Sequential(
            nn.Linear(53*53*16,120),
            nn.Linear(120,84),
            nn.Linear(84,2),
            nn.Softmax(dim=1),
            )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1,53*53*16)       # Change if you modify network
        x = self.cls(x)
        return x

model = trainmodel()
model.to(device)

trainmodel(
  (conv): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (cls): Sequential(
    (0): Linear(in_features=44944, out_features=120, bias=True)
    (1): Linear(in_features=120, out_features=84, bias=True)
    (2): Linear(in_features=84, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)

In [ ]:
## test your model sequential output
conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
    )
batch = torch.rand(3,3,224,224)    
result = conv(batch) #It is your img input
print(result.shape)



torch.Size([3, 16, 53, 53])


## 2.3 How to improve Network?

### Layer
- Adjustment of convolution/pooling layer
- [Activation Layer](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)
- Global [Average](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html#torch.nn.AdaptiveAvgPool2d)/[Max](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveMaxPool2d.html#torch.nn.AdaptiveMaxPool2d) Pooling

### Training Robustness
- [Augmentation](https://pytorch.org/vision/stable/transforms.html)
- [Batch Normalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html#torch.nn.BatchNorm2d)
- [Dropout Layers](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html#torch.nn.Dropout)

### Optimizer: attempt to find global minimum
- [SGD with momentum](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)
- [ADAM](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam)
- [torch.optim](https://pytorch.org/docs/stable/optim.html)

### Learning Rate Sceduler
- [Cosine Annealing Learning Rate](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.CosineAnnealingLR.html#torch.optim.lr_scheduler.CosineAnnealingLR)


In [ ]:
##############################################
# Build your model here!
# 
# Practice:
#   Add data augmentation !
##############################################

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # Todo
    transforms.ToTensor(),
    ])

trainset.transform = train_transform
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers = 2)

In [ ]:
##############################################
# Build your model here!
# 
# Practice:
#   Improve your own Model!
##############################################

class trainmodel(nn.Module):
    def __init__(self):
        super(trainmodel, self).__init__()
        nn.Conv2d(3,128,3),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        ...
        nn.AdaptiveAvgPool2d(output_size = 8)

    def forward(self, x):
        nn.Linear(64*8*8,2)
        nn.Softmax(dim=1)
        return x

model = trainmodel()
model.to(device)


# 3 Training Routine

## 3.1 Training module

In [ ]:
def train(model, trainloader, optimizer, criterion):
    # keep track of training loss
    train_loss = 0.0
    train_correct = 0
    
    # train the model
    model.train()
    for data, target in tqdm(trainloader):
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        # update training Accuracy
        _, predicted = torch.max(output.data, 1)
        train_correct += (predicted == target).sum().item()

    return train_loss/len(trainloader.dataset), train_correct/len(trainloader.dataset)

In [ ]:
@torch.no_grad()
def test(model, testloader, criterion):
    # keep track of validation loss
    valid_loss = 0.0
    valid_correct = 0

    # evaluate the model 
    model.eval()
    for data, target in tqdm(testloader):
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        # update validation Accuracy
        _, predicted = torch.max(output.data, 1)
        valid_correct += (predicted == target).sum().item()

    return valid_loss/len(testloader.dataset), valid_correct/len(testloader.dataset)

In [ ]:
def modeltrain(model, trainloader, validloader, testloader, optimizer, criterion, epochs, save_model_path, earlystop=4):
    history = {
        'trainloss' : [],
        'trainacc' : [],
        'validloss' : [],
        'validacc' : [],
    }
    state = {
        'epoch' : 0,
        'state_dict' : model.state_dict(),
        'trainloss' : 10000,
        'trainacc' : 0,
        'validloss' : 10000,
        'validacc' : 0,
    }
    valid_loss_min = 10000
    trigger = 0
    for epoch in range(epochs):
        print(f'running epoch: {epoch+1}')
        trainloss, trainacc = train(model, trainloader, optimizer, criterion)
        validloss, validacc = test(model, validloader, criterion)

        # print training/validation statistics 
        history['trainloss'].append(trainloss)
        history['trainacc'].append(trainacc)
        history['validloss'].append(validloss)
        history['validacc'].append(validacc)
        print(f'Training Loss  : {trainloss:.4f}\t\tTraining Accuracy  : {trainacc:.4f}')
        print(f'Validation Loss: {validloss:.4f}\t\tValidation Accuracy: {validacc:.4f}')
        
        # save model if validation loss has decreased
        if validloss <= valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.4f} --> {validloss:.4f}).  Saving model ...\n')
            state['epoch'] = epoch
            state['state_dict'] = model.state_dict()
            state['trainloss'] = trainloss
            state['trainacc'] = trainacc
            state['validloss'] = validloss
            state['validacc'] = validacc

            torch.save(state, save_model_path)
            valid_loss_min = validloss
            trigger = 0
        # if model dont improve for 5 times, interupt.
        else:
            trigger += 1
            print(f'Validation loss increased ({valid_loss_min:.4f} --> {validloss:.4f}). Trigger {trigger}/{earlystop}\n')
            if trigger == earlystop:
                break
    print('\nTest Evaluate:')
    testloss, testacc = test(model, testloader, criterion)
    state['testloss'] = testloss
    state['testacc'] = testacc
    torch.save(state, save_model_path)
    bestepoch = state['epoch']
    validloss = state['validloss']
    validacc = state['validacc']
    print(f'Best model on epoch : {bestepoch}/{epoch}')
    print(f'validation loss: {validloss:.4f}\t\t validation acc : {validacc:.4f}')
    print(f'test loss      : {testloss:.4f}\t\t test acc \t: {testacc:.4f}')
    return history

## 3.2 Start Training!

In [ ]:
from tqdm.notebook import tqdm
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
n_epochs = 4
save_model_path = os.path.join(SavePath, 'model_weight.pth')

history = modeltrain(
        model = model,
        trainloader = trainloader,
        validloader = validloader,
        testloader = testloader,
        optimizer = optimizer,
        criterion = criterion,
        epochs = n_epochs,
        save_model_path = save_model_path
        )

running epoch: 1


  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

Training Loss  : 0.6532		Training Accuracy  : 0.6214
Validation Loss: 0.6418		Validation Accuracy: 0.6510
Validation loss decreased (10000.0000 --> 0.6418).  Saving model ...

running epoch: 2


  0%|          | 0/110 [00:00<?, ?it/s]

...

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

Training Loss  : 0.5627		Training Accuracy  : 0.7374
Validation Loss: 0.5875		Validation Accuracy: 0.7020
Validation loss decreased (0.6065 --> 0.5875).  Saving model ...


Test Evaluate:


  0%|          | 0/47 [00:00<?, ?it/s]

Best model on epoch : 3/3
validation loss: 0.5875		 validation acc : 0.7020
test loss      : 0.5779		 test acc 	: 0.7210


## 3.3 Plot the result

In [ ]:
import json
save_history = json.dumps(history)
with open(os.path.join(SavePath, 'history.json'), 'w') as f:
    json.dump(save_history, f)

In [ ]:
import matplotlib.pyplot as plt

def plot(name, savedir, trainhistory, validhistory):
    plt.figure(figsize=(10,5))
    plt.plot(trainhistory, label = 'train')
    plt.plot(validhistory,  label = 'valid')
    plt.title(name)
    plt.xlabel("epochs")
    plt.show()
    plt.savefig(savedir)

In [ ]:
plot('Training Loss', os.path.join(SavePath,'loss.png'), history['trainloss'], history['validloss'])

In [ ]:
plot('Training Accuracy', os.path.join(SavePath,'acc.png'), history['trainacc'], history['validacc'])

# 4 Inference testing data

### 4.1 load weight

In [ ]:
## create model as same as your training 
model = trainmodel()
model.to(device)

## load weight
state = torch.load(save_model_path)
# state['state_dict']
model.load_state_dict(state['state_dict'])

### 4.2 test and save result

In [ ]:
transform = transforms.Compose([
       transforms.Resize((224, 224)),
       transforms.ToTensor(),
])

class test_set(Dataset):
    def __init__(self, img_path, csv_path, transform = transform):
        # --------------------------------------------
        # Initialize paths, transforms, and so on
        # --------------------------------------------
        self.df = pd.read_csv(csv_path)
        self.img_path = img_path
        self.transform = transform
        
    def __getitem__(self, index):
        # --------------------------------------------
        # 1. Read from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data (e.g. image and label)
        # --------------------------------------------
        imgpath = os.path.join(self.img_path, self.df.iloc[index, 0])
        img = Image.open(imgpath)
        img = self.transform(img)

        return img, index
        
    def __len__(self):
        # --------------------------------------------
        # Indicate the total size of the dataset
        # --------------------------------------------
        return len(self.df)

In [ ]:
import torch.nn.functional as F

#load test data
testimg_path = os.path.join(DataPath, 'test_dataset')
testcvs_path = os.path.join(DataPath, 'test.csv')

df = pd.read_csv(testcvs_path)

Class = {'cat':0, 'dog':1}
inv_Class = dict((v, k) for k, v in Class.items())

print(len(df))

testset = test_set(testimg_path, testcvs_path)
testloader = DataLoader(testset, batch_size=1, shuffle=False,pin_memory=True,num_workers = 2)

model.eval()
with torch.no_grad():
    for data, idx in tqdm(testloader):
        data = data.to(device)
        pred = F.softmax(model(data),dim=1)
        pred = np.argmax(pred.detach().cpu().numpy(),axis=1)

        df.at[idx, 'label'] = inv_Class[int(pred)] # convert predicted integer back to class string, only works when batch = 1

df.head()
df.to_csv(os.path.join(SavePath, 'result.csv'), encoding='utf-8')

In [ ]:
## Only examples have "test_ans.csv"
df = pd.read_csv(os.path.join(SavePath, 'test_ans.csv'))
ans = df["label"].to_numpy()

df = pd.read_csv(SavePath+'/result.csv')
output = df["label"].to_numpy()

print("Accuracy:",((ans == output).sum().item()/len(df)))

# 5 What Next? [Transfer Learning](https://hackmd.io/@allen108108/H1MFrV9WH)


<p align="center">
<img src="https://miro.medium.com/max/1400/1*Ww3AMxZeoiB84GVSRBr4Bw.png" width="700">
</p>

### 5.1 Transfer Learning on Pytorch 
[Transfer learning for computer vision tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

[Models and pre-trained weights](https://pytorch.org/vision/stable/models.html)


In [ ]:
"""
import torchvision.models as models

resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet_v2 = models.mobilenet_v2(pretrained=True)
mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)
mobilenet_v3_small = models.mobilenet_v3_small(pretrained=True)
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)
efficientnet_b0 = models.efficientnet_b0(pretrained=True)
efficientnet_b1 = models.efficientnet_b1(pretrained=True)
efficientnet_b2 = models.efficientnet_b2(pretrained=True)
efficientnet_b3 = models.efficientnet_b3(pretrained=True)
efficientnet_b4 = models.efficientnet_b4(pretrained=True)
efficientnet_b5 = models.efficientnet_b5(pretrained=True)
efficientnet_b6 = models.efficientnet_b6(pretrained=True)
efficientnet_b7 = models.efficientnet_b7(pretrained=True)
regnet_y_400mf = models.regnet_y_400mf(pretrained=True)
regnet_y_800mf = models.regnet_y_800mf(pretrained=True)
regnet_y_1_6gf = models.regnet_y_1_6gf(pretrained=True)
regnet_y_3_2gf = models.regnet_y_3_2gf(pretrained=True)
regnet_y_8gf = models.regnet_y_8gf(pretrained=True)
regnet_y_16gf = models.regnet_y_16gf(pretrained=True)
regnet_y_32gf = models.regnet_y_32gf(pretrained=True)
regnet_x_400mf = models.regnet_x_400mf(pretrained=True)
regnet_x_800mf = models.regnet_x_800mf(pretrained=True)
regnet_x_1_6gf = models.regnet_x_1_6gf(pretrained=True)
regnet_x_3_2gf = models.regnet_x_3_2gf(pretrained=True)
regnet_x_8gf = models.regnet_x_8gf(pretrained=True)
regnet_x_16gf = models.regnet_x_16gf(pretrained=True)
regnet_x_32gf = models.regnet_x_32gf(pretrained=True)
vit_b_16 = models.vit_b_16(pretrained=True)
vit_b_32 = models.vit_b_32(pretrained=True)
vit_l_16 = models.vit_l_16(pretrained=True)
vit_l_32 = models.vit_l_32(pretrained=True)
convnext_tiny = models.convnext_tiny(pretrained=True)
convnext_small = models.convnext_small(pretrained=True)
convnext_base = models.convnext_base(pretrained=True)
convnext_large = models.convnext_large(pretrained=True)

"""